<b><h2 style="text-align:center;">COMM493 - Coding AI for Business</h2><b>

<h5 style="text-align:center;">Assignment 1 - Linear Learner Data Identification & Preparation</h5>

<h5 style="text-align:center;">Maxwell Brookes - 20244724</h5>

<h5 style="text-align:center;">Due on Feb 9</h5>


<b><h4>About The Dataset:</h4><b>
<h5>asdjklasj klasjkl fjaklsdjfklsjadklf jalskfjklsd j fasdklj fklasjdf ljaslfjsdlfj kadj</h5>


<b><h4>Load Modules & Tools:</h4><b>

In [1]:
import numpy as np  # for linear algebra
import pandas as pd  # for loading data
# from matplotlib import pyplot as plt  # for visualizations
import sys  # for system information

print(sys.version)
print("Successfully Loaded All Modules")

3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
Successfully Loaded All Modules



<b><h4>Load Dataset:</h4><b>

In [2]:
# dataset = https://www.kaggle.com/datasets/smac91/hotel-bookings, implementation = https://www.kaggle.com/datasets/somnathnaskar/hotel-booking-dataset/data
df = pd.read_csv('hotel_bookings.csv')  # Load the CSV file into a DataFrame


<b><h4>Preview Dataset:</h4><b>

In [3]:
print('Data Shape:', df.shape)  # print the shape of the data
print('Data Preview:')
print(df.iloc[:10, :5])  # list first 10 row and columns

Data Shape: (119390, 32)
Data Preview:
          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month
0  Resort Hotel            0        342               2015               July
1  Resort Hotel            0        737               2015               July
2  Resort Hotel            0          7               2015               July
3  Resort Hotel            0         13               2015               July
4  Resort Hotel            0         14               2015               July
5  Resort Hotel            0         14               2015               July
6  Resort Hotel            0          0               2015               July
7  Resort Hotel            0          9               2015               July
8  Resort Hotel            1         85               2015               July
9  Resort Hotel            1         75               2015               July



<b><h4>Drop Columns:</h4><b>
<h5>'agent' and 'company' because it doe snot matter in this context</h5>
<h5>'arrival_date_week_number' because 'arrival_date_month' and 'arrival_date_day_of_month' are already included</h5>
<h5>'country' because analysis is not based on location</h5>
<h5>'market_segment' because the values don't provide significant insight</h5>
<h5>'distribution_channel' because it has overlap with 'market_segment'</h5>
<h5>'days_in_waiting_list' because most values are zero</h5>
<h5>'meal' because most values are BB, which implies that all hotels offer the same</h5>
<h5>'reservation_status_date' because the info is already captured with 'lead_time'</h5>
<h5>'reservation_status' conveys the same information as 'is_cancelled'</h5>
<h5>'reserved_room_type' and 'assigned_room_type' because a small kingston business will not have many room options</h5>h5

In [4]:
# print(df.columns)  # Check available columns
# unique_dict = {col: df[col].unique() for col in df.columns}
# for col, uniques in unique_dict.items():
#     print(f"{col}: {uniques}")
columns_to_drop = ['agent', 'company', 'arrival_date_week_number',
                   'country', 'market_segment', 'distribution_channel',
                   'days_in_waiting_list', 'meal', 'reservation_status',
                  'reserved_room_type', 'assigned_room_type'] # maybe add is_repeated_guest 
df.drop(columns_to_drop, axis=1, inplace=True)  # drop columns without having to create a new pandas data frame
print('Data Shape:', df.shape)

Data Shape: (119390, 21)



<b><h4>Further Cleaning (Rows, Unrealistic, Outliers, ...):</h4><b>

In [5]:
df.drop_duplicates(keep='first', inplace=True)  # Drop duplicate rows, keeping only the first occurrence
df['children'].fillna(0, inplace=True)  # Replace NaN values in 'children' with 0

df.drop(df[df['hotel'] == 'Resort Hotel'].index, inplace=True)  # Drop all rows where 'hotel' is 'Resort Hotel'
df.drop(['hotel'], axis=1, inplace=True)  # Drop the 'hotel' column since only 'City Hotel' remains

df.drop(df[df['adults'] == 0].index, inplace=True)  # Drop rows where 'adults' is 0
df.drop(df[df['adults'] > 15].index, inplace=True)  # Drop rows where 'adults' is greater than 15
df.drop(df[(df['children'] > 3) | (df['babies'] > 2)].index, inplace=True)  # Drop rows where 'children' > 3 or 'babies' > 2

df.drop(df[df['lead_time'] > 500].index, inplace=True)  # Drop rows where 'lead_time' is greater than 500
df.drop(df[(df['stays_in_week_nights'] > 30) | (df['stays_in_weekend_nights'] > 10)].index, inplace=True)  # Drop rows where 'stays_in_week_nights' > 30 or 'stays_in_weekend_nights' > 10

df.reset_index(drop=True, inplace=True)  # Reset the index after all modifications

print('Data Shape:', df.shape)

Data Shape: (50804, 20)
